# 3-way cross-task geometry: ODR 1.5s, ODR 3.0s, ODRd

Compare Procrustes geometry across all three tasks using only the
**4 overlapping cardinal directions** (0/90/180/270) and a
**common delay window** (500–1700 ms) that precedes the ODRd distractor.

- ODR columns `[0, 2, 4, 6]` = ODRd columns `[0, 1, 2, 3]`
- Common epochs: cue (0–500 ms), delay (500–1700 ms)
- Features per neuron: 4 directions × 2 epochs = 8
- Common monkeys: OLI, PIC, ROS, UNI
- ODRd data: raw 4-column (pools all distractor variants as trials)

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os, sys

sys.path.insert(0, '.')
from functions import (
    load_cardinal_task_data, filter_common_monkeys,
    compute_flat_tuning, cross_task_cv, plot_cross_task,
    print_cross_task_summary,
)

DATA_DIR      = '../data_raw'
N_PCS         = 8
MIN_NEURONS   = N_PCS + 1
BIN_MS        = 50
COMMON_EPOCHS = {'cue': (0, 500), 'delay': (500, 1700)}
N_ITER        = 100
SEED          = 42

## 1. Load data & filter to common monkeys

In [ ]:
cardinal_data = load_cardinal_task_data(DATA_DIR)
tasks, common_monkeys = filter_common_monkeys(cardinal_data)
print(f'Common monkeys: {common_monkeys}')

for name in tasks:
    print(f'{name}: {len(tasks[name]["ids"])} neurons, {tasks[name]["data"].shape[1]} dirs')

print('\nNeurons per monkey:')
for mid in common_monkeys:
    counts = {t: np.sum(tasks[t]['ids'] == mid) for t in tasks}
    print(f'  {mid}: ' + ', '.join(f'{t}={c}' for t, c in counts.items()))

## 2. Tuning curves (cue + common delay)

In [ ]:
T_RANGES = {'ODR 1.5s': (-1000, 2500), 'ODR 3.0s': (-1000, 3500), 'ODRd': (-1000, 3500)}

tuning_flat = {}
for name in tasks:
    flat, _, _ = compute_flat_tuning(tasks[name]['data'], T_RANGES[name], COMMON_EPOCHS, BIN_MS)
    tuning_flat[name] = flat
    print(f'{name}: {flat.shape} (neurons x features)')

## 3. Split-half cross-validation

In [4]:
task_ids = {name: tasks[name]['ids'] for name in tasks}
results = cross_task_cv(tuning_flat, task_ids, N_PCS, MIN_NEURONS, N_ITER, SEED)

  iteration 25/100
  iteration 50/100
  iteration 75/100
  iteration 100/100
Done.


## 4. Results and distance matrix

In [ ]:
plot_cross_task(results)
print_cross_task_summary(results)